**Unimos dos CSV para ampliar el número de datos sobre el factor Birth Attended By Skilled Person. Calculamos la media de los que no sean el mismo valor**

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Cargamos los CSVs en DataFrames
df1 = pd.read_csv('7_BirthAttendedBySkilledPersonal1.csv',sep=',')
df2 = pd.read_csv('7_BirthAttendedBySkilledPersonal2.csv',sep=',')

In [3]:
#Eliminamos columnas innecesarias
df1.drop('Indicator', axis=1, inplace=True)
df2.drop('Series Name', axis=1, inplace=True)

In [4]:
#Cambiamos el tipo de dato de Value y eliminamos los NaN
df2['Value'] = pd.to_numeric(df2['Value'], errors='coerce')
df2.dropna(inplace=True)

In [5]:
#Cambiamos el nombre de las columnas
df1.columns = ['Country Name','Time', 'Value1']

In [7]:
#Cambiamos el nombre de los países para que coincidan

changeNames = [('Bahamas, The','Bahamas'), ('Cabo Verde', 'Cape Verde'),
               ( 'Congo, Dem. Rep.','Democratic Republic of Congo'), ('Congo, Rep.', 'Congo'),
               ('Czech Republic','Czechia'), ("Côte d'Ivoire","Cote d'Ivoire"), ('Côte d’Ivoire',"Cote d'Ivoire"), 
               ("Democratic People's Republic of Korea",'North Korea'), 
               ('Democratic Republic of the Congo', 'Democratic Republic of Congo'),  ('Ethiopia PDR', 'Ethiopia'), 
               ('Federated States of Micronesia','Micronesia (country)'), ('Gambia, The', 'Gambia'), 
               ('Iran (Islamic Republic of)', 'Iran'), ('Iran, Islamic Rep.', 'Iran'), 
               ('Korea, Dem. People’s Rep.', 'North Korea'), ('Korea, Rep.', 'South Korea'), ('Lao PDR', 'Laos'),
               ("Lao People's Democratic Republic", 'Laos'), ('Micronesia (Federated States of)', 'Micronesia (country)'),
               ('Micronesia, Fed. Sts.', 'Micronesia (country)'), ('Republic of Korea', 'South Korea'),
               ('Republic of Moldova', 'Moldova'), ('Russian Federation', 'Russia'), ('Slovak Republic', 'Slovakia'),
               ('State of Palestine', 'Palestine'), ('Syrian Arab Republic', 'Syria'), ('The Bahamas', 'Bahamas'), 
               ('The Gambia', 'Gambia'), ('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom'),
               ('United Republic of Tanzania', 'Tanzania'), ('United States of America', 'United States'),
               ('Venezuela (Bolivarian Republic of)', 'Venezuela'), ('Venezuela, RB', 'Venezuela'), ('Viet Nam', 'Vietnam'),
               ('Yemen, Rep.', 'Yemen')]

for df in [df1,df2]:
    for oldName, newName in changeNames:
        df.loc[df['Country Name'] == oldName, 'Country Name'] = newName

In [8]:
#Vemos los países que difieren en ambos dataframes
set(df1['Country Name'].unique()).difference(set(df2['Country Name'].unique()))

{'Andorra',
 'Bolivia (Plurinational State of)',
 'Cook Islands',
 'Egypt',
 'Greece',
 'Kyrgyzstan',
 'Niue',
 'Palestine',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Sudan (until 2011)',
 'The former Yugoslav Republic of Macedonia'}

In [9]:
#Mergeamos ambos DFs
df_completo = pd.merge(df1, df2, on=['Country Name', 'Time'], how='outer')
df_completo.insert(0, 'Series Name', 'Births attended by skilled health personnel (%)')
df_completo = df_completo.sort_values(['Country Name', 'Time']).reset_index(drop=True)

In [10]:
#Unimos en una columna los valores y calculamos el valor medio de aquellos que difieran
df_completo['Value_mean'] = np.nan
listaDiff = []
for i in range(len(df_completo)):
    if(np.isnan(df_completo['Value'][i])):
        df_completo['Value_mean'][i] = df_completo['Value1'][i]
    elif(np.isnan(df_completo['Value1'][i])):
        df_completo['Value_mean'][i] = df_completo['Value'][i]
    else:
        df_completo['Value_mean'][i] = (df_completo['Value'][i] + df_completo['Value1'][i]) / 2
        if(df_completo['Value'][i] - df_completo['Value1'][i] != 0):
            listaDiff.append(abs(df_completo['Value'][i] - df_completo['Value1'][i]))

<ipython-input-10-17c893d8415a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo['Value_mean'][i] = df_completo['Value'][i]
<ipython-input-10-17c893d8415a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo['Value_mean'][i] = (df_completo['Value'][i] + df_completo['Value1'][i]) / 2
<ipython-input-10-17c893d8415a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo['Value_mean'][i] = df_completo['Value1']

In [11]:
#Vemos la diferencia entre los valores que no coinciden(151 valores)
print('Número de valores que difieren: ', len(listaDiff))
listaDiff

Número de valores que difieren:  180


[0.5999999999999943,
 3.0,
 0.030000000000001137,
 0.04000000000000625,
 0.04000000000000625,
 1.7999999999999972,
 8.0,
 7.200000000000003,
 2.5,
 0.30000000000001137,
 1.7000000000000028,
 2.5,
 0.20000000000000284,
 9.100000000000001,
 0.09999999999999432,
 0.09999999999999432,
 0.20000000000000284,
 1.5999999999999943,
 3.8999999999999915,
 1.1000000000000085,
 2.3999999999999915,
 6.400000000000006,
 0.10000000000000853,
 0.10000000000000853,
 0.5,
 1.2000000000000028,
 1.5999999999999943,
 0.10000000000000853,
 0.29999999999999716,
 0.19999999999998863,
 0.19999999999998863,
 0.19999999999998863,
 0.10000000000000853,
 0.10000000000000853,
 0.10000000000000853,
 0.10000000000000853,
 0.7000000000000028,
 13.0,
 1.2000000000000028,
 1.5,
 1.5999999999999943,
 4.100000000000001,
 0.04999999999999716,
 0.030000000000001137,
 0.04999999999999716,
 0.030000000000001137,
 0.04999999999999716,
 6.800000000000011,
 4.8999999999999915,
 5.700000000000003,
 3.1999999999999886,
 3.200000000

In [12]:
#Eliminar columnas innecesarias y cambiar nombre
df_completo.drop(['Value1', 'Value'], axis=1, inplace=True)
df_completo.columns = ['Indicator', 'Country', 'Year', 'Value']

In [13]:
#Lo almacenamos en un CSV
df_completo.to_csv('7_BirthAttendedBySkilledPersonal.csv',sep=',', index=False)